In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1,2,3,4,5,6'

In [ ]:
# %run -p ../train_cnf_adaptive.py --data mnist --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 5000 --save ../experiments_published/cnf_published_bs8K_errcontrol_7gpus_adaptive_1_2 --seed 0 --conditional False --controlled_tol True --lr 0.001 --warmup_iters 56 --atol 1e-4  --rtol 1e-4 --log_freq 11 --batch_size_schedule "2-3-4-5-6-7-8-9-10-11"
# #

In [ ]:
%run -p ../train_cnf.py --data mnist --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 8000 --test_batch_size 5000 --save ../experiments_published/cnf_published_bs8K_errcontrol_7gpus_adaptive_1_2 --resume ../experiments_published/cnf_published_bs8K_errcontrol_7gpus_adaptive_1_2/epoch_400_checkpt.pth --seed 0 --conditional False --controlled_tol True --lr 0.0001 --warmup_iters 113 --atol 1e-4  --rtol 1e-4
#

/tancode/repos/tan-ffjord/train_cnf.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWriter

# go fast boi!!
torch.backends.cudnn.benchmark = True
SOLVERS = ["dopri5", "bdf", "rk4", "midpoint", 'adams', 'explicit_adams']
parser = argparse.ArgumentParser("Continuous Normalizing Flow")
parser.add_argument("--data", choices=["mn